In [1]:
import torch

CACHE_DIR = '/data/huangjie'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 扩散模型数据集构建  
**基本数据标注**：1、基本基于开源模型进行出发去对数据进行标注；2、最大限度的压榨设备（可能就需要考虑到使用多进程的方式）；  
**进阶数据标注**：此部分可能就需要去考虑到一些论文里面对于数据构建的算法，然后将算法提取出来进行运用  
**注意事项**：1、对于算法需要分析各类模型，比如说YOLO算法有几种，可能就需要分析各类YOLO对于显存的占用；2、最后的所有的标签格式最好是要**符合标准的标记格式**

## 使用BLIP2生成简单的文本匹配

In [ ]:
import torch
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16
).to(device)

# 输入图片（可以替换为本地图片路径或 URL）
img_path_or_url = "https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg"  # 示例图片 URL
# 如果使用本地图片，替换为：img_path_or_url = "path/to/your/image.jpg"

# 加载和预处理图片
if img_path_or_url.startswith("http"):
    raw_image = Image.open(requests.get(img_path_or_url, stream=True).raw).convert("RGB")
else:
    raw_image = Image.open(img_path_or_url).convert("RGB")

# 预处理图片并生成描述
inputs = processor(images=raw_image, return_tensors="pt").to(device, torch.float16)
generated_ids = model.generate(**inputs, max_new_tokens=30)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

# 输出生成的文本描述
print(f"Generated Caption: {generated_text}")

## 使用Yolo+SAM2切割图片

## 使用VLM模型生成图片文本描述